In [ ]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd
import sys
import glob
import time

import tensorflow as tf
from tensorflow.keras.losses import mse
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers.legacy import Adam

from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score

In [ ]:
def get_number_features(df, number_features):
    
    columns = list(df.columns)
    chosen_columns = []
    
    if number_features == 92:
        for column in columns:
            if column.find('L5') != -1 or column.find('L3') != -1 or column.find('L1') != -1 or column.find('L=0.1') != -1:
                chosen_columns.append(column)
        df = pd.DataFrame(df, columns = chosen_columns)
        return df
    elif number_features == 69:
        for column in columns:
            if column.find('L5') != -1 or column.find('L3') != -1 or column.find('L1') != -1:
                chosen_columns.append(column)
        df = pd.DataFrame(df, columns = chosen_columns)
        return df
    elif number_features == 46:
        for column in columns:
            if column.find('L5') != -1 or column.find('L3') != -1:
                chosen_columns.append(column)
        df = pd.DataFrame(df, columns = chosen_columns)
        return df
    elif number_features == 23:
        for column in columns:
            if column.find('L5') != -1:
                chosen_columns.append(column)
        df = pd.DataFrame(df, columns = chosen_columns)
        return df
    else:
        return df

In [ ]:
def get_files(path, number_features):
    
    all_files = glob.glob(os.path.join(path , '*.csv'))

    files_list = []
    for file in all_files:
        df = pd.read_csv(file, index_col = None, encoding = 'utf-8', sep = ',', low_memory = False)
        files_list.append(df)
        
    df = pd.concat(files_list, axis = 0, ignore_index = True)
    df = get_number_features(df, number_features)
    
    return df

In [ ]:
def get_files_botiot(path):
    
    all_files = glob.glob(os.path.join(path , '*.csv'))

    files_list = []
    for file in all_files:
        df = pd.read_csv(file, index_col = None, encoding = 'utf-8', sep = ',', low_memory = False)
        files_list.append(df)
        
    df = pd.concat(files_list, axis = 0, ignore_index = True)
    
    # Drop irrelevant information
    df = df.drop(columns=['pkSeqID', 'stime', 'flgs', 'flgs_number', 'saddr', 'sport', 'daddr', 'dport', 'subcategory', 'category'])
    
    # Categorical to numerical
    df['proto'] = df['proto'].map({'tcp': 1, 'arp': 2, 'udp': 3, 'icmp': 4, 'ipv6-icmp': 5})
    df['state'] = df['state'].map({'REQ': 1, 'RST': 2, 'ACC': 3, 'CON': 4, 'INT': 5, 'URP': 6, 'FIN': 7, 'NRS': 8, 'ECO': 9, 'TST': 10, 'MAS': 11})
    
    df_benign = df
    df_attack = df
    
    # Get only benign data
    df_benign = df_benign.query('attack == 0')
    df_benign = df_benign.drop(columns=['attack'])
    
    # Get only the attacks
    df_attack = df_attack.query('attack == 1')
    df_attack = df_attack.drop(columns=['attack'])
        
    return df_benign, df_attack